In [1]:
import tensorflow as tf
import numpy as np
from SOE_Net_model_full import *
import init_SOE_NET as init_net
import configure as cfg

In [2]:
tf.compat.v1.enable_eager_execution()

In [5]:
import time
start_time = time.time()


vol = np.random.rand(1, 42, 100, 100, 1)
vol = tf.constant(vol, dtype=tf.float32)
orientations = np.array(init_net.initOrientations (cfg.ORIENTATIONS,cfg.SPEEDS,cfg.NUM_DIRECTIONS), dtype=np.float32)
basis = init_net.initSeparableFilters('basis', cfg.FILTER_TAPS, filter_type="G3")
# biases_soe = init_net.initBiases('bias', 0)
SOE_Net(vol, basis, orientations, 0)

print("--- %s seconds ---" % (time.time() - start_time))

(1, 42, 100, 100, 20)
(1, 42, 100, 100, 40)
(1, 21, 50, 50, 80)
(1, 7, 17, 17, 160)
(1, 3200)
--- 4.07513689994812 seconds ---


# Channel wise pooling

In [6]:
vol = np.random.rand(1, 42, 10, 10, 1600)
vol = tf.constant(vol, dtype=tf.float32)

In [7]:
NUM_ORIENTATION = 10

In [8]:
num_ch = vol.get_shape().as_list()[-1]
num_block = num_ch // (NUM_ORIENTATION ** 2)
pooled = []
for i in range(num_block):
    vol_slice = vol[:,:,:,:,i*(NUM_ORIENTATION ** 2): (i + 1)*(NUM_ORIENTATION ** 2)]
    for j in range(NUM_ORIENTATION):
        pooled.append(tf.reduce_mean(vol_slice[:,:,:,:,j::NUM_ORIENTATION], axis=-1, keepdims=True))
output = tf.concat(pooled, axis=-1)

In [9]:
output.get_shape()

TensorShape([Dimension(1), Dimension(42), Dimension(10), Dimension(10), Dimension(160)])

In [10]:
num_ch = vol.get_shape().as_list()[-1]
l = []
for i in range(NUM_ORIENTATION):
    x = np.zeros((NUM_ORIENTATION ** 2, 1), dtype=np.float32)
    x[i::NUM_ORIENTATION] = 1.0 / NUM_ORIENTATION
    l.append(x)
chunk = np.concatenate(l, axis=-1)
f = np.zeros((num_ch, num_ch // NUM_ORIENTATION), dtype=np.float32)
for i in range(num_ch // (NUM_ORIENTATION ** 2)):
    f[i * (NUM_ORIENTATION ** 2): (i + 1) * (NUM_ORIENTATION ** 2), i * NUM_ORIENTATION: (i + 1) * NUM_ORIENTATION] = chunk

output1 = tf.nn.conv3d(vol, tf.constant(f[None, None, None, :, :], dtype=tf.float32), strides=[1, 1, 1, 1, 1], padding='VALID')


2022-09-01 17:58:35.820110: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0


In [11]:
output1.get_shape()

TensorShape([Dimension(1), Dimension(42), Dimension(10), Dimension(10), Dimension(160)])

In [12]:
tf.math.reduce_max(output - output1)

<tf.Tensor: id=2058, shape=(), dtype=float32, numpy=1.7881393e-07>

In [25]:
# def CP3D(name, in_data):
#     num_ch = vol.get_shape().as_list()[-1]
#     cc_filter = CPP(name, cfg.NUM_DIRECTIONS, num_ch, cfg.REC_STYLE)
#     print(cc_filter.get_shape())
#     if cfg.REC_STYLE == 'two_path':
#         shape_L1 = 2*cfg.NUM_DIRECTIONS
#     else:
#         shape_L1 = cfg.NUM_DIRECTIONS
    
#     if in_data.shape[-1] == shape_L1:
#         cc_pool = in_data    
#     else:
#         cc_pool = tf.nn.conv3d(in_data, cc_filter, strides=[1, 1, 1, 1, 1], padding='VALID')
#     return cc_pool

In [26]:
# def CPP(name, num_dir, num_ch, rec_style):
#     if rec_style == 'two_path':
#         l = []
#         for i in range(num_dir):
#             x = np.zeros((num_dir ** 2, 1), dtype=np.float32)
#             x[i::num_dir] = 1.0 / num_dir
#             l.append(x)
#         chunk = np.concatenate(l, axis=-1)
#         f = np.zeros((num_ch, num_ch // num_dir), dtype=np.float32)
#         for i in range(num_ch // (num_dir ** 2)):
#             f[i * (num_dir ** 2): (i + 1) * (num_dir ** 2), i * num_dir: (i + 1) * num_dir] = chunk
#         return tf.constant(f[None, None, None, :, :], dtype=tf.float32)
#     else:
#         pass

In [27]:
output2 = CP3D("a", vol)

(1, 1, 1, 1600, 160)


In [29]:
tf.math.reduce_max(output1 - output2)

<tf.Tensor: id=2067, shape=(), dtype=float32, numpy=0.0>